In [24]:
import pandas as pd
import os
import chardet
import re
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Set up directories
graphs_dir = os.path.join('graphs')
os.makedirs(graphs_dir, exist_ok=True)

# Detect encoding of a file
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        return chardet.detect(f.read(10000))['encoding']

# Extract year from filename
def extract_year_from_filename(filename):
    match = re.search(r'(\d{4})\.csv', filename)
    return int(match.group(1)) if match else None

# Function to generate and save histogram with gradient colors
def generate_and_save_histogram(data, year):
    # Define bins for average_score (up to 200 with step 20)
    bins = range(100, 201, 20)  # From 100 to 200 with step 20
    filtered_data = data[data['average_score'] <= 200]  # Limit to 200
    
    plt.figure(figsize=(10, 6))
    n, bins, patches = plt.hist(filtered_data['average_score'], bins=bins, edgecolor='white', linewidth=1.5)
    
    # Create a soft gradient colormap
    colors = ['#A3BFFA', '#96CEB4', '#D4A5A5', '#FFD3B6', '#FFAAA5', '#CDB4DB']  # Soft pastel colors
    cmap = LinearSegmentedColormap.from_list('custom_gradient', colors, N=len(patches))
    for i, patch in enumerate(patches):
        patch.set_facecolor(cmap(i / len(patches)))
        patch.set_edgecolor('white')
        patch.set_linewidth(1.5)
    
    # Enhance design with simple year title
    plt.title(f'{year}', fontsize=16, pad=15, fontweight='bold', color='#333333')
    plt.xlabel('Average Score Range', fontsize=12, color='#555555')
    plt.ylabel('Count', fontsize=12, color='#555555')
    plt.grid(True, alpha=0.1, linestyle='--', linewidth=0.5, color='#CCCCCC')
    plt.xticks(bins, fontsize=10, color='#666666')
    plt.yticks(fontsize=10, color='#666666')
    plt.gca().set_facecolor('#F5F6F5')
    
    # Save as SVG
    output_path = os.path.join(graphs_dir, f'average_score_histogram_{year}.svg')
    plt.savefig(output_path, format='svg', bbox_inches='tight', transparent=True)
    plt.close()

# Function to generate and save line plot for multiple years
def generate_and_save_line_plot(data_by_year):
    years = sorted(data_by_year.keys())
    if len(years) < 6:
        print("⚠️ Недостатньо років для 6 ліній. Знайдено тільки:", years)
        return
    
    plt.figure(figsize=(12, 6))
    
    # Soft colors for lines
    line_colors = ['#A3BFFA', '#96CEB4', '#D4A5A5', '#FFD3B6', '#FFAAA5', '#CDB4DB']
    
    for i, year in enumerate(years[:6]):  # Limit to 6 years
        data = data_by_year[year][data_by_year[year]['average_score'] <= 200]
        bins = range(100, 201, 20)  # Ensure range up to 200
        hist, bin_edges = np.histogram(data['average_score'], bins=bins, density=True)
        # Use full bin_edges including the last point
        plt.plot(bin_edges, np.append(hist, 0), marker='o', label=f'{year}', linewidth=2, color=line_colors[i % len(line_colors)], markeredgecolor='white', markersize=6)
    
    # Enhance design with simple title
    plt.title('Average Scores Distribution', fontsize=16, pad=15, fontweight='bold', color='#333333')
    plt.xlabel('Average Score', fontsize=12, color='#555555')
    plt.ylabel('Density', fontsize=12, color='#555555')
    plt.grid(True, alpha=0.1, linestyle='--', linewidth=0.5, color='#CCCCCC')
    plt.xticks(range(100, 201, 20), fontsize=10, color='#666666')
    plt.yticks(fontsize=10, color='#666666')
    plt.legend(title='', fontsize=10, frameon=True, facecolor='#F5F6F5', edgecolor='white', loc='best')
    plt.gca().set_facecolor('#F5F6F5')
    
    # Save as SVG
    output_path = os.path.join(graphs_dir, 'average_score_line_plot.svg')
    plt.savefig(output_path, format='svg', bbox_inches='tight', transparent=True)
    plt.close()

# Process data and generate histograms and line plot
def process_and_generate_histograms(input_folder="filtered_data", output_folder="aggregated_data"):
    os.makedirs(output_folder, exist_ok=True)
    data_by_year = {}

    available_years = set()
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            year = extract_year_from_filename(filename)
            if year:
                available_years.add(year)

    if not available_years:
        print("⚠️ Не знайдено файлів із роками в назвах")
        return

    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            input_path = os.path.join(input_folder, filename)
            year = extract_year_from_filename(filename)
            if year not in available_years:
                continue
            print(f"🔄 Обробка {filename} (рік: {year})...")
            
            try:
                encoding = detect_encoding(input_path)
                df = pd.read_csv(input_path, encoding=encoding, low_memory=False)
                
                initial_count = len(df)
                filtered_df = df[
                    (df['subjects_count'] >= 3) & 
                    (df['total_score'] > 0) & 
                    (df['average_score'] > 100)
                ]
                new_count = len(filtered_df)
                
                if new_count == 0:
                    print(f"⚠️ У файлі {filename} немає даних після фільтрації")
                    continue

                filtered_df['year'] = year
                data_by_year[year] = filtered_df

                # Generate and save histogram
                generate_and_save_histogram(filtered_df, year)
                print(f"✅ Збережено гістограму для року {year} у {graphs_dir}/average_score_histogram_{year}.svg")
                print(f"   Залишено рядків: {new_count}/{initial_count} ({new_count/initial_count:.1%})")

            except Exception as e:
                print(f"❌ Критична помилка у файлі {filename}: {str(e)}")

    # Generate and save line plot
    if data_by_year:
        generate_and_save_line_plot(data_by_year)
        print(f"✅ Збережено лінійний графік у {graphs_dir}/average_score_line_plot.svg")

    print("\n🎉 Обробку та генерацію гістограм/графіка завершено! Результати збережено у папці 'graphs'")

# Run the process
if __name__ == "__main__":
    process_and_generate_histograms()

🔄 Обробка 2020.csv (рік: 2020)...
✅ Збережено гістограму для року 2020 у graphs/average_score_histogram_2020.svg
   Залишено рядків: 201212/201212 (100.0%)
🔄 Обробка 2021.csv (рік: 2021)...
✅ Збережено гістограму для року 2021 у graphs/average_score_histogram_2021.svg
   Залишено рядків: 188609/188609 (100.0%)
🔄 Обробка 2023.csv (рік: 2023)...
✅ Збережено гістограму для року 2023 у graphs/average_score_histogram_2023.svg
   Залишено рядків: 256313/256313 (100.0%)
🔄 Обробка 2022.csv (рік: 2022)...
✅ Збережено гістограму для року 2022 у graphs/average_score_histogram_2022.svg
   Залишено рядків: 213647/213647 (100.0%)
🔄 Обробка 2019.csv (рік: 2019)...
✅ Збережено гістограму для року 2019 у graphs/average_score_histogram_2019.svg
   Залишено рядків: 172734/172734 (100.0%)
🔄 Обробка 2024.csv (рік: 2024)...
✅ Збережено гістограму для року 2024 у graphs/average_score_histogram_2024.svg
   Залишено рядків: 264164/264164 (100.0%)
✅ Збережено лінійний графік у graphs/average_score_line_plot.svg

## Box Plots

In [26]:
import pandas as pd
import os
import chardet
import re
import matplotlib.pyplot as plt

graphs_dir = os.path.join('graphs')
os.makedirs(graphs_dir, exist_ok=True)

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        return chardet.detect(f.read(10000))['encoding']

def extract_year_from_filename(filename):
    match = re.search(r'(\d{4})\.csv', filename)
    return int(match.group(1)) if match else None

def generate_and_save_box_plot(data_by_year):
    years = sorted(data_by_year.keys())
    if len(years) < 6:
        print("⚠️ Недостатньо років для 6 бокс-плотів. Знайдено тільки:", years)
        return
    
    plt.figure(figsize=(12, 6))
    
    data_for_box = [data_by_year[year]['average_score'].dropna() for year in years[:6]]
    
    blue_shades = ['#A3BFFA', '#87CEEB', '#ADD8E6', '#B0E0E6', '#AFEEEE', '#B0C4DE']
    
    plt.boxplot(data_for_box, labels=[str(year) for year in years[:6]], patch_artist=True)
    for patch, color in zip(plt.gca().artists, blue_shades):
        patch.set_facecolor(color)
        patch.set_edgecolor('white')
        patch.set_linewidth(1.5)
    
    plt.title('Box Plots of Average Scores', fontsize=16, pad=15, fontweight='bold', color='#333333')
    plt.xlabel('Year', fontsize=12, color='#555555')
    plt.ylabel('Average Score', fontsize=12, color='#555555')
    plt.grid(True, alpha=0.1, linestyle='--', linewidth=0.5, color='#CCCCCC')
    plt.xticks(fontsize=10, color='#666666')
    plt.yticks(range(100, 201, 20), fontsize=10, color='#666666')
    plt.gca().set_facecolor('#F5F6F5')
    
    output_path = os.path.join(graphs_dir, 'average_score_box_plot.svg')
    plt.savefig(output_path, format='svg', bbox_inches='tight', transparent=True)
    plt.close()

def process_and_generate_histograms(input_folder="filtered_data", output_folder="aggregated_data"):
    os.makedirs(output_folder, exist_ok=True)
    data_by_year = {}

    available_years = set()
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            year = extract_year_from_filename(filename)
            if year:
                available_years.add(year)

    if not available_years:
        print("⚠️ Не знайдено файлів із роками в назвах")
        return

    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            input_path = os.path.join(input_folder, filename)
            year = extract_year_from_filename(filename)
            if year not in available_years:
                continue
            print(f"🔄 Обробка {filename} (рік: {year})...")
            
            try:
                encoding = detect_encoding(input_path)
                df = pd.read_csv(input_path, encoding=encoding, low_memory=False)
                
                initial_count = len(df)
                filtered_df = df[
                    (df['subjects_count'] >= 3) & 
                    (df['total_score'] > 0) & 
                    (df['average_score'] > 100)
                ]
                new_count = len(filtered_df)
                
                if new_count == 0:
                    print(f"⚠️ У файлі {filename} немає даних після фільтрації")
                    continue

                filtered_df['year'] = year
                data_by_year[year] = filtered_df

                print(f"✅ Оброблено {filename} для року {year}")
                print(f"   Залишено рядків: {new_count}/{initial_count} ({new_count/initial_count:.1%})")

            except Exception as e:
                print(f"❌ Критична помилка у файлі {filename}: {str(e)}")

    if data_by_year:
        generate_and_save_box_plot(data_by_year)
        print(f"✅ Збережено бокс-плот у {graphs_dir}/average_score_box_plot.svg")

    print("\n🎉 Обробку та генерацію бокс-плотів завершено! Результати збережено у папці 'graphs'")

if __name__ == "__main__":
    process_and_generate_histograms()

🔄 Обробка 2020.csv (рік: 2020)...
✅ Оброблено 2020.csv для року 2020
   Залишено рядків: 201212/201212 (100.0%)
🔄 Обробка 2021.csv (рік: 2021)...
✅ Оброблено 2021.csv для року 2021
   Залишено рядків: 188609/188609 (100.0%)
🔄 Обробка 2023.csv (рік: 2023)...
✅ Оброблено 2023.csv для року 2023
   Залишено рядків: 256313/256313 (100.0%)
🔄 Обробка 2022.csv (рік: 2022)...
✅ Оброблено 2022.csv для року 2022
   Залишено рядків: 213647/213647 (100.0%)
🔄 Обробка 2019.csv (рік: 2019)...
✅ Оброблено 2019.csv для року 2019
   Залишено рядків: 172734/172734 (100.0%)
🔄 Обробка 2024.csv (рік: 2024)...
✅ Оброблено 2024.csv для року 2024
   Залишено рядків: 264164/264164 (100.0%)


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_3481/648143087.py:30: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data_for_box, labels=[str(year) for year in years[:6]], patch_artist=True)


✅ Збережено бокс-плот у graphs/average_score_box_plot.svg

🎉 Обробку та генерацію бокс-плотів завершено! Результати збережено у папці 'graphs'
